export cells as modules?



In [ ]:
var fs = require('fs');
var path = require('path');
var rimraf = require('rimraf');
var importer = require('../Core');
var mkdirpSync = importer.import('mkdirp');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var MODULE_OUTPUT = __dirname + '/.modules';

function getExtension(cell) {
    var extension;
    if(cell.cell_type === 'markdown') {
        extension = '.md';
    } else if(cell.cell_type === 'raw') {
        extension = '.txt';
    } else if(cell.language === 'javascript') {
        if(cell.source.join('').match(/describe\s*\(/igm)) {
            extension = '.spec.js'
        } else {
            extension = '.js';
        }
    } else if(cell.language === 'powershell') {
        extension = '.ps1';
    } else if(cell.language === 'csharp') {
        extension = '.cs';
    } else if(cell.language === 'python') {
        extension = '.py';
    } else if(cell.language === 'typescript') {
        if(cell.source.join('').match(/describe\s*\(/igm)) {
            extension = '.spec.ts'
        } else {
            extension = '.ts';
        }
    } else if(cell.language === 'bash') {
        extension = '.sh';
    } else {
        throw 'unknown language or cell type: ' + notebook + ' (' + cell.cell_type + ',' + cell.language + ')';
    }
    return extension;
}

function exportNotebook(notebook, project) {
    if(fs.existsSync(MODULE_OUTPUT)) {
        rimraf.sync(MODULE_OUTPUT);
    }
    const name = path.basename(notebook).replace(/\.ipynb/ig, '');
    const parent = path.resolve(path.dirname(notebook)).replace(path.resolve(project), '');
    mkdirpSync(path.join(MODULE_OUTPUT, parent, name));
    
    return importer.getCells(notebook, ['*', 'code', 'markdown', 'raw'])
        .reduce((results, cell, i) => {
            const extension = getExtension(cell);
            // TODO: get previous markdown and extract name that leads back to the current cell
            const cellPath = path.join(MODULE_OUTPUT, parent, name, 'cell-' + i + extension);
            fs.writeFileSync(cellPath, cell.source.join(''));
            results.push(cellPath);
            return results;
        }, [])
}
module.exports = exportNotebook;


export all notebooks?

Export all notebooks to a structured folder where each cell has it's own file for linting?



In [ ]:
var glob = require('glob');
var path = require('path');
var importer = require('../Core');
var exportNotebook = importer.import('notebook.ipynb[export cells modules]');

function exportAll(project) {
    const notebooks = glob.sync('**/*.ipynb', {cwd: project});
    return notebooks.map(n => {
        return exportNotebook(path.join(project, n), project)
    })
}
module.exports = exportAll;


test notebook export?



In [ ]:
var path = require('path');
var glob = require('glob');
var assert = require('assert');
var fs = require('fs');
var importer = require('../Core');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = path.resolve(__dirname + '/../');
var MODULE_OUTPUT = PROJECT_PATH + '/Utilities/.modules';

var sortNumeric = (that) => {
    return that.sort((a, b) => parseInt(a.split(/[^0-9]/ig).join('')) - parseInt(b.split(/[^0-9]/ig).join('')));
}

describe('notebook export service', () => {
    it('should export this file', () => {
        exportNotebook(path.resolve(PROJECT_PATH + '/Utilities/notebook.ipynb'));
        var files = glob.sync('Utilities/notebook/*', {cwd: MODULE_OUTPUT});
        sortNumeric(files);
        assert(path.basename(files[0]) === 'cell-0.md');
        assert(path.basename(files[1]) === 'cell-1.js');
        assert(path.basename(files[2]) === 'cell-2.md');
    })
    
    it('should have comparable cells', () => {
        var importCells = importer.interpret(['notebook.ipynb[0]', 'notebook.ipynb[1]', 'notebook.ipynb[2]']);
        exportNotebook(path.resolve(PROJECT_PATH + '/Utilities/notebook.ipynb'));
        var files = glob.sync('Utilities/notebook/*', {cwd: MODULE_OUTPUT});
        sortNumeric(files);
        var fsCell0 = fs.readFileSync(path.join(MODULE_OUTPUT, files[0])).toString();
        var fsCell1 = fs.readFileSync(path.join(MODULE_OUTPUT, files[1])).toString();
        var fsCell2 = fs.readFileSync(path.join(MODULE_OUTPUT, files[2])).toString();
        assert(fsCell0 === importCells[0].markdown.join(''));
        assert(fsCell1 === importCells[0].fresher, fsCell1.length + ' != ' + importCells[0].fresher.length)
        assert(fsCell2 === importCells[1].markdown.join(''))
    })
    
    it('should export all notebooks', () => {
    //    var exported = exportAll(PROJECT_PATH);
    //    assert(exported.length > 0);
    })
})


import all cell modules?

import files back in to cells? (two-way workflow)


In [ ]:
var fs = require('fs');
var glob = require('glob');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = PROFILE_PATH + '/Documents/jupytangular2/Utilities/.modules';
var project = PROFILE_PATH + '/Documents/jupytangular2';

var cells = glob.sync('**/cell-*', {cwd: PROJECT_PATH});
for(const c of cells) {
    const cell = path.basename(c);
    const notebook = path.basename(path.dirname(c));
    const parent = path.basename(path.dirname(path.dirname(c)));
    if(parent === 'jupytangular2') {
        continue;
    }
    const nb = JSON.parse(fs.readFileSync(path.join(project, parent, notebook + '.ipynb')));
    let counter = 0;
    for(const i in nb.cells) {
        if(!nb.cells.hasOwnProperty(i)) {
            continue;
        }
        // TODO: reimport all cells
        if(nb.cells[i].cell_type === 'code') {
            if(cell === 'cell-' + counter + '.js') {
                nb.cells[i].source = fs.readFileSync(path.join(PROJECT_PATH, c)).toString().split('\n');
            }
            counter++;
        }
    }
    fs.writeFileSync(path.join(project, parent, notebook + '.ipynb'), JSON.stringify(nb, null, 2));
}


In [ ]:
// TODO: delete .modules if everything checks out

// TODO: import tests


TODO:

import module as notebook cell?

import gist as notebook

import random instructions as notebook

rules:
- testing cells have zero markdown before them,
- all public functions are made public using module.export or global namespace
- use describe( test blocks to describe parsing and replacement information for the functions it is testing, i.e. function utility(root) would have a describe block describe('utility(root)') neatly formated and parsed for calendar commands
- every file is one feature
- every cell or export should be a single purpose component with proper includes/dependencies
- every cell must be under 100 lines
- every function must be testable insolation
- every function should be made accesible from the command line using module.exports
- every function should be runnable from notebooks using typeof $$ !== 'undefined'



utility function to export notebooks and deploy to lambda

In [9]:
// TODO: generalize this for all notebooks?

// assuming you've already run `aws configure`
var fs = require('fs');
var os = require('os');
var path = require('path');
var importer = require('../Core');
var {
    execCmd,
    delint,
    exportNotebook
} = importer.import([
    'spawn child process',
    'delinting notebooks',
    'export cells as modules'
]);

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var PROJECT_PATH = path.join(PROFILE_PATH, '/Documents/asm/subscription.services/Subscription.Services.EloquaImport');

function regexToArray(ex, str, i = 0) {
    var co = []; var m;
    while ((m = ex.exec(str)) && co.push(m[i])) ;
    return co;
};

function replaceImports(cell, exports) {
    var newCell = cell
        .replace(/.*require\('\.\.\/Core'\).*\s*/igm, '')
        .replace(/.*typeof \$\$[\s\S]*?\}/igm, '')
        .replace(/\bvar\b/igm, 'let');
    
    const imports = regexToArray(/import\([\['"]([\s\S]*?)['"\]]\)/igm, newCell, 1);
    console.log(imports);
    imports.forEach(i => {
        const result = importer.interpret(i);
        const mapping = Object.keys(exports).filter(k => result.id === exports[k].id);
        if(mapping.length > 0) {
            newCell = newCell.replace(new RegExp('importer.import\\([\[\'"]' + i + '[\'"\]]\\)', 'igm'), 'require(\'.' + mapping[0] + '\')');
        } else {
            throw new Error('unmatched import! ' + result.id);
        }
    })
    return newCell;
}

function exportAndDeploy(notebook) {
    const exports = exportNotebook(notebook, '..');
    exports.forEach(e => {
        if(e.indexOf('.js') > -1) {
            const cell = replaceImports(fs.readFileSync(e).toString(), exports);
            fs.writeFileSync(path.join(PROJECT_PATH, path.basename(e)), cell);
        }
    })
    // TODO: zip and upload to AWS
    //return delint(PROJECT_PATH)
}
module.exports = exportAndDeploy;

if(typeof $$ !== 'undefined') {
    $$.async();
    exportAndDeploy('../Frameworks/zuora to eloqua.ipynb')
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


[ 'request polyfill', 'map zuora data to eloqua fields' ]


Error: unmatched import! polyfills.ipynb[0]